# Uczenie głębokie cz. II

Ten notatnik ma na celu przedstawienie sposobów wykorzystania uczenia głębokiego w różnych zastosowaniach. W trakcie zadania najpierw dostosujemy istniejącą sieć konwolucyjną do nowych danych, zobaczymy jak stosować sieci do rekomendowania produktów oraz jak nauczyć klasyfikator tekstowy. Na tych zajęciach wykorzystamy bibliotekę [fast.ai](https://github.com/fastai/fastai), która jest wrapperem na [PyTorch](https://pytorch.org/) oraz, tak jak ostatnio, [Keras](https://keras.io/). Poniższe skrytpty są w istocie kompilacją różnych materiałów szkoleniowych do fast.ai i Kerasa.

Po wykonaniu tego zadania powinieneś:
+ wiedzieć dostosować istniejącą architekturę (wraz z wagami) do własnego problemu, 
+ wiedzieć jak zamrażać i odmrażać warstwy sieci,
+ umieć zastosować sieć neuronową do danych tekstowych,
+ potrafić stowrzyć własny model rekomendacyjny oparty o sieć neuronową.

## Przygotowanie środowiska na Crestle

Ćwiczenie wykonamy na platformie [Crestle](https://www.crestle.com), aby nie musieć instalować bibliotek i zbiorów danych lokalnie. 

Po zalogowaniu na Crestle wgraj tego notebooka do folderu

```
/courses/fastai/
```

Gdy wgrasz notebooka, zacznij od odpalenia poniższej komendy, aby korzystać z najnowszej wersji fast.ai. fast.ai wciąż jest w wersji alfa, więc najlepiej korzystać bezpośrednio z wersji na githubie.

In [ ]:
!git pull

Ponieważ przykłady są trudne i uczenie sieci trwa długo (nawet na GPU) każdy wykona tylko jedno z poniższych zadań:
- **Rozpoznawanie obrazów**
- **Tworzenie systemu rekomendacyjnego**
- **Analiza tekstu**

W chwilach, gdy sieć się uczy i jest parę minut wolnego możesz zajrzeć do sąsiada, który wykonuje inne zadanie i wymienić się uwagami.

## Rozpoznawanie obrazów

Spróbujemy nauczyć sieć rozpoznającą typy terenu na zdjęciach satelitarnych. W tym celu wykorzystamy sieć nauczoną na zbiorze danych ImageNet, która umie klasyfikaować pojedyncze obiekty na zdjęciach (psy, koty, czołgi, itp.). W naszym problemie będziemy musieli rozpoznać kilka rodzajów terenu na każdym zdjęciu (multi-label classification) i nie będą one przypominały obiektów ze zbioru ImageNet.

### Dane

Dane pochądzą z zakończonego konkursu na Kaggle o nazwie [Planet: Understanding the Amazon from Space](https://www.kaggle.com/c/planet-understanding-the-amazon-from-space). Dane są już wgrane na naszej instancji na Crestle. Musimy tylko odpowiednio ułożyć dane w foldery.

Wykorzystamy opcję wywoływania poleceń systemowych w Jupyter (`!<komenda>`) do utworzenia linków symbolicznych.

In [ ]:
%matplotlib inline
import os
PATH = 'data/planet/'

os.makedirs('data/planet/models', exist_ok=True)
os.makedirs('/cache/planet/tmp', exist_ok=True)

!ln -s /datasets/kaggle/planet-understanding-the-amazon-from-space/train-jpg {PATH}
!ln -s /datasets/kaggle/planet-understanding-the-amazon-from-space/test-jpg {PATH}
!ln -s /datasets/kaggle/planet-understanding-the-amazon-from-space/train_v2.csv {PATH}
!ln -s /cache/planet/tmp {PATH}

Jeśli porządki się udały zobaczmy jak wyglądają dwa przykładowe zdjęcia satelitarne.

In [ ]:
from fastai.plots import *

list_paths = [f"{PATH}train-jpg/train_0.jpg", f"{PATH}train-jpg/train_1.jpg"]
titles=["haze primary", "agriculture clear primary water"]
plots_from_files(list_paths, titles=titles, maintitle="Multi-label classification")

Zdjęcie po lewej ma etykiety (klasy) *haze* i *primary*, a zdjęcie po prawej ma etykiety *agriculture*, *clear*, *primary* i *water*.

### Dostosowywanie sieci

W pierwszej kolejności ustalimy sposób ładowania danych do sieci i miarę oceny, którą będziemy śledzić.

In [ ]:
from fastai.conv_learner import *
from sklearn.metrics import fbeta_score
import warnings

label_csv = f'{PATH}train_v2.csv' # plik csv z połączeniem nazwa_obrazu-etykiety
n = len(list(open(label_csv)))-1 # liczba przykładów
val_idxs = get_cv_idxs(n) # numery przykładów które weźmiemy do oceny krzyżowej

**Zad. 1.1: Sprawdź ile przykładów ma cały zbiór danych i ile z tych przykładów zostało odłożonych jako przykłady walidacyjne.**

Teraz czas na miarę oceny. W konkursie liczona była miara F2. To prawie to samo co F1-score, o którym kiedyś wspominaliśmy, tylko precision i recall mają nierówne wagi w średniej harmonicznej.

Choć sama miara może Cię nie interesować, to zwróć uwagę, że możesz śledzić wynik dowolnej funkcji, która przyjmuje dwa parametry:
- `preds` predykcje,
- `targs` prawdziwe etykiety.

In [ ]:
def f2(preds, targs, start=0.17, end=0.24, step=0.01):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        return max([fbeta_score(targs, (preds>th), 2, average='samples')
                    for th in np.arange(start,end,step)])
    
metrics = [f2]

Teraz najważniejsze. Nie będziemy podawać do sieci naszych obrazów od tak. Będziemy dane losowo modywfikować. Operacja ta po angielsku nosi nazwę *data augmentation* i w przypadku zdjęć z reguły obejmuje:
- obroty
- pomniejszanie obrazów
- zoom
- przesunięcia lewo/prawo
- lustrzane odbicia

Takie transformacje kryją się w bibliotece fast.ai pod akronimem `tfms` (od transformations). Funkcja `tfms_from_model` jest odpowiedzialna za wykonywanie transformacji oraz przystosowywanie sieci do nowych danych, czyli inicjalizację oraz normalizację wag wejściowych przy pierwszym uczeniu.

Na koniec za pomocą klasy `ImageClassifierData` tworzymy generator danych wejściowych do sieci (tzw. DataModel). Już kiedyś o tym wspominaliśmy, ale w ramach przypomnienia: generator w Pythonie to iterator, czyli w praktyce funkcja która zamiast zwracać całą listę iobiektów od razu, potrafi to robić po jednym obiekcie naraz.

**Zad. 1.2: Poniżej stworzymy metodę, która będzie generowała dane o zadanym rozmiarze `sz`. Spróbuj określić jakie inne transformacje będą wykonywane na obrazach.**

In [ ]:
f_model = resnet34

def get_data (sz):
    tfms = tfms_from_model(f_model, sz, aug_tfms=transforms_top_down, max_zoom=1.05)
    return ImageClassifierData.from_csv(PATH, 'train-jpg', label_csv, tfms=tfms,
                    suffix='.jpg', val_idxs=val_idxs, test_name='test-jpg')

Teraz przechodzimy do wykorzystania istniejącej sieci. fast.ai zawiera szereg gotowych architektur takich VGG, Resnet, Resnext. Można skorzystać z samej architektury lub z architektury wraz ze wstępnie wyuczonymi wagami (ang. *pretrained*). Jak widać w bloku powyżej, my skorzystamy z architektury resnet34, która jest jedną z nowszych i szybszych architektur.

### Uczenie

Skorzystamy z architektury wraz ze wstępnie wyuczonymi wagami. Taki wstępnie nauczony model ma zamrożone wagi. fast.ai sam dodaje za nas ostatnie warstwy, które będą stanowić model i będą odmrożone.

Aby przyspieszyć proces dostosowywania sieci można skorzystać z następującej sztuczki. Zmniejszymy obrazy do rozmiaru 64X64 żeby sieć była mniejsza i uczenie szło szybciej. Żeby jeszcze przyspieszyć początkowe uczenie wygenerowane dane (zmniejszone, lustrzane odbicia, itd.) zapiszemy do osobnych plików (w folderze `tmp`), żeby nie tracić czasu na generowanie ich w locie.

In [ ]:
sz=64
data = get_data(sz)
data = data.resize(int(sz*1.3), 'tmp')

Teraz magia fast.ai: prosimy o wstępnie wyuczony model (`ConvLearner.pretrained`), o zadanej architekturze (`f_model`), do przetwarzania naszych danych (`data`) i śledzenia wskazanego przez nas zbioru miar (`metrics`). Zwróć uwagę, że nie definiujemy żadnych warstw, funkcji aktywacji, optymalizatora czy funkcji straty. fast.ai pozwala na ustawienie tych rzeczy, ale celem tej biblioteki jest zaoferowanie dobrych wartości domyślnych i ustawienie większości parametrów za nas.

In [ ]:
learn = ConvLearner.pretrained(f_model, data, metrics=metrics)

Ostatnią rzeczą, którą musimy zrobić to określić learning rate. fast.ai promuje heurystykę do znajdowania tej wartości. W skrócie: przetestuj wiele różnych learning rate i wybierz tę wartość, która powoduje największe tempo obniżania wartości funkcji straty.

Odpal poniższy kod, przyjrzyj się wykresowi i prównaj z wartością `lr` w kolejnej komórce.

In [ ]:
lrf=learn.lr_find()
learn.sched.plot()

In [ ]:
lr = 0.2

OK, wybraliśmy learning rate, teraz przechodzimy do faktycznego uczenia. Odpalamy `fit` podając jako parametry:
- liczbę cykli uczenia
- liczbę epok w pierwszym cyklu
- ile razy dłuższe mają być kolejne cykle

W naszym przypadku mamy trzy cykle, każdy dwa razy dłuższy, czyli: 1 + 2 + 4 = 7. Ta sztuczka z tworzeniem cykli uczenia nazywa się Stochastic Gradient Descent with Restarts (SGDR) i możecie o niej poczytać np. [tutaj](https://arxiv.org/abs/1608.03983).

Dość gadania. Zacznijmy uczyć sieć.

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

Teraz odmrozimy (`unfreeze`) wcześniejsze warstwy sieci i nadamy osobne learning rate (`lrs`) dla każdego z trzech fragmentów sieci resnet.

Zwróć uwagę, że najmniej chcemy zmieniać wagi w początkowych warstwach sieci (`lr/9`) a najbardziej w ostatnich (`lr`). Pierwsze warstwy we wstępnie wyuczonej sieci konwolucyjnej wykrywają takie cechy jak krawędzie czy kolory i wag w tych warstwach nie chcemy zbytnio zmieniać. Ostatnie warstwy to z kolei nasz model, który musimy uczyć od zera stąd potrzeba szybszego tempa zmian.

In [ ]:
learn.unfreeze()
lrs = np.array([lr/9,lr/3,lr])

learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)
learn.save(f'{sz}')

Zwróć uwagę, że zapisaliśmy model do pliku. To taki środek bezpieczeństwo gdybyśmy musieli z jakiegoś powodu przerwać uczenie na jakiś czas. Oprócz możliwości zapisu do pliku, obiekt learn ma szereg innych przydatnych informacji. Możemy np. narysować krzywą uczenia.

In [ ]:
learn.sched.plot_loss()

A teraz zwiększymy obrazy do 128x128 i powtórzymy tę samą sztuczkę. Najpierw douczymy ostatnie warstwy, a potem odmrozimy wcześniejsze i zaktualizujemy im wagi.

In [ ]:
sz=128

learn.set_data(get_data(sz))
learn.freeze()
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)

In [ ]:
learn.unfreeze()
learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)
learn.save(f'{sz}')

**Zad. 1.3: Powtórz powyższą operację dla obrazów w pełnej rozdzielczości, czyli 256x256.**

### Keras

Podobną metodologię, obejmującą generowanie sztucznych danych wraz z wykorzystaniem istniejącej architektury i wag, można oczywiście zaimplementować również w Kerasie. Krótki tutorial jak to zrobić znajdziesz na stronie: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html.

## System rekomendacyjny

Sieci neuronowe są tak ogólnym narzędziem, że można je wykorzystać do wielu różnych zadań. Jednym z takich zadań, o którym jeszcze nie mówiliśmy, jest rekomendowanie produktów. Jednym ze sztandarowych algorytmów do tego problemu jest [collaborative filtering](https://en.wikipedia.org/wiki/Collaborative_filtering).

Ogólna idea algorytmu to estymacja oceny produktu przez klienta na podstawie:
- podobieństwa tego klienta do innych osób które już oceniły dany produkt,
- podobieństwa tego produktu do innych produktów, które ocenił klient.

W najprostzej formie sprowadza się to do ważenia wektora klientów i produktów w taki sposób by uzyskać oceny. Poprosimy, żeby fast.ai zrobił dla nas coś podobnego z wykorzystaniem sieci neuronowych do określenia wag produktów i klientów.

### Przygotowanie danych

Zaczniemy od ściągnięcia zbioru danych. Do ćwiczenia wykorzystamy podzbiór recenzji filmów ze strony [movielens](https://movielens.org/). Odpal poniższy kod, aby załadować dane do folderu `data` na Crestle.

In [ ]:
!wget -O ./data/ml-latest-small.zip http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

Teraz rozpakujemy zbiór danych.

In [ ]:
!unzip ./data/ml-latest-small.zip -d ./data

OK. Teraz zaimportujemy parę bibliotek, które będą nam potrzebne i zobaczmy jak wygląda zbiór uczący.

In [ ]:
%matplotlib inline

from fastai.learner import *
from fastai.column_data import *
import seaborn as sns

path = 'data/'
ratings = pd.read_csv(path + 'ratings.csv')
ratings.head()

Super. Mamy identyfikatory filmów i użytkownków oraz oceny od 1 do 5. Ponieważ będziemy chcieli śledzić proces uczenia się sieci, określimy, że część przykładów będzie zbiorem walidacyjnym. W tym celu skorzystamy z funkcji `get_cv_idxs` z biblioteki fast.ai.

**Zad. 2.1: Sprawdź ile przykładów ma cały zbiór danych i ile z tych przykładów zostało odłożonych jako przykłady walidacyjne.**

In [ ]:
val_idxs = get_cv_idxs(len(ratings))

### Uczenie modelu

Nie będziemy musieli dużo robić żeby stworzyć mdel przewidujący oceny filmów. Musimy tylko określić kilka parametrów. W ramach ćwiczenia od razu podamy dość dobre wartości, ale normalnie trzeba by było dobrać je metodą prób i błędów.

In [ ]:
wd=2e-4        # weight decay, czyli stała regularyzacji - parametr pilnujący żeby sieć nie przeuczyła się za szybko
n_factors = 50 # głębokość embeddingu, czyli za pomocą ilu liczb (wag) wyrazimy jednego użytkownika czy jeden film
batch=64       # grupa przykładów, które przepuszczamy przez sieć między etapami propagacji wstecznej

Teraz stworzymy sieć. W fast.ai definicja sieci jest powiązana ze zbiorem danych (nazywają to `DataModel`). My skorzystamy z `CollabFilterDataset` i stworzymy taki model w oparciu o plik csv. W kolejnych prametrach podamy, które kolumny odpowiadają za klientów (użytkowników), produkty (filmy) i ocenę.

Następnie w opaciu o DataModel poprosimy fast.ai o sieć, którą zapiszemy to zmiennej `learn`.

In [ ]:
cf = CollabFilterDataset.from_csv(path, 'ratings.csv', 'userId', 'movieId', 'rating')
learn = cf.get_learner(n_factors, val_idxs, batch, opt_fn=optim.Adam)

Ostatni krok to puszczenie uczenia.

**Zad. 2.2: Spróbuj określić co oznaczają kolejne parametry metody `fit()`.**

In [ ]:
learn.fit(1e-2, 2, wds=wd, cycle_len=1, cycle_mult=2)

**Zad. 2.3: Wynik który dostałeś (ok. 0.776) to MSE (funkcja straty, którą za Ciebie określił fast.ai). Wylicz RMSE, aby oszacować o ile średnio model myli się w swojej ocenie.**

Na koniec zwizualizujemy sobie predykcje. Wykorzystamy do tego bibliotekę seaborn, którą możesz kojarzyć z jednych z wcześniejszych zajęć.

In [ ]:
preds = learn.predict()

y=learn.data.val_y
sns.jointplot(preds, y, kind='hex', stat_func=None);

**Zad. 2.4: Zmień zwiększ batch size i liczbę epok (cykli). Porównaj wyniki.**

## Przetwarzanie języka naturalnego

Spróbujemy teraz stworzyć sieć do przewidywania tematyki tekstu. Przyjmiemy następującą strategię:
- najpierw skorzystamy z gotowych word embeddings (tych samych o których mówiliśmy parę zajęć temu),
- stworzymy sieć, w której word embeddings będą tworzyć periwszą warstwę,
- nauczymy tak sklejoną sieć przewidywać jedną z 20 klas w zbiorze uczącym.

To zadanie wykonamy wyjątkowo w Keras. Tak jak zwykle, parę bibliotek na początek.

In [ ]:
%matplotlib inline

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant

### Przygotowanie danych

Jako word embedding wykorzystamy model GloVe oparty o 6 miliardów tekstów. Oficjalną stroną dla tego modelu jest: https://nlp.stanford.edu/projects/glove/.

Jako zbiór danych do naszych eksperymentów wykorzystamy zbiór `newsgroup`. Jest to dosyć stary zbiór danych posiadający teksty obejmujące 20 różnych tematów. Zbiór nie jest zbyt duży dzięki czemu jest szansa nauczyć model nawet lokalnie.

Poniższy kod zawiera ścieżki, w których będziemy oczekiwać embeddings i danych. Ponadto ustawimy kilka parametrów, które wykorzystamy później.

In [ ]:
BASE_DIR = './data/'
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')

MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

A teraz ściągnijmy potrzebne dane.

In [ ]:
!wget -O ./data/glove.6B.zip http://nlp.stanford.edu/data/glove.6B.zip
!wget -O ./data/news20.tar.gz http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz

Rozkakujmy paczkę z word embeddings. Zajmie sporo miejsca, ale powinna rozpakować się dosyć szybko.

In [ ]:
!unzip ./data/glove.6B.zip -d ./data/glove.6B

Warto wspomnieć, że w Internecie można znaleźć gotowe word embeddings dla różnych języków. Poniżej pierwsze dwa trafienia dla języka niemieckiego:
- https://devmount.github.io/GermanWordEmbeddings/
- https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md

Dobra. Teraz przeiterujemy przez cały zestaw word embeddings i stworzymy **słownik**, gdzie kluczem będzie **słowo** a wartością wektor liczb reprezentujących **embedding**.

In [ ]:
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Super, mamy %s embeddingsów.' % len(embeddings_index))

**Zad. 3.1: Ile liczb jest powiązanych z jednym słowem w ramach jednego word embedding?**

Teraz rozpakujemy zbiór danych newsgroup. To niestety potrwa dosyć długo - choć zbiór nie jest taki duży, to składa się z wielu tysięcy plików.

In [ ]:
!tar -xzf ./data/news20.tar.gz -C ./data # to trochę potrwa

Teraz przejedziemy przez zbiór danych i wyłuskamy listę tekstów oraz wektor etykiet.

In [ ]:
texts = []  # lista tekstów (atrybuty opisowe)
labels_index = {}  # słownik mapujący klasę na liczbę (identyfikator klasy)
labels = []  # lista etykiet (atrybut decyzyjny) 

for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}
                with open(fpath, **args) as f:
                    t = f.read()
                    i = t.find('\n\n') 
                    if 0 < i:
                        t = t[i:]
                    texts.append(t)
                labels.append(label_id)

print('Mamy %s tekstów.' % len(texts))

Tak się składa, że Keras oferuje własny tokenizator. Pozwoli on nam podzielić tekst na słowa i zachować tylko najpopularniejsze wyrazy (`MAX_NUM_WORDS`). Najperw odpalimy `fit`, żeby określić najpopularniejsze słowa, a następnie faktycznie stokenizujemy teksty za pomocą `texts_to_sequences`.

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Mamy %s unikatyowych tokenów.' % len(word_index))

Ponieważ większość sieci lubi wejście w konkretnym rozmiarze, skorzystamy z funkcji `pad_sequences` aby ustandaryzować wejście. Za długie sekwencje słów przytniemy, a za krótkie uzupełnimy putymi znakami.

In [ ]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Jako ostatni krok przygotowania, standardowe czynności:
- kodowanie etykiet w sposób binarny,
- podział na dane uczące i walidacyjne.

In [ ]:
labels = to_categorical(np.asarray(labels))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

### Uczenie

Teraz po tym całym przetwarzaniu danych spróbujemy stworzyć klasyfikator. Najpierw wykorzystamy nasze word embeddings z Glove żeby stworzyć pierwszą warstwę sieci. Ponieważ ograniczyliśmy słownik do `MAX_NUM_WORDS`, nasza warstwa będzie miała co najwyżej tyle wag.

Pozostaje nam tylko zmapować identyfikator słowa z jego embedding, żeby zainicjalizować wagi.

In [ ]:
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

Teraz stworzymy własną sieć. Zwróć uwagę, że korzystamy z 1-wymiarowych konwolucji, czyli "filtra" który przesuwa się po kolejnych grupach słów. Tak dla przypomnienia konwolucje 2D przesuwały kwadratowy filtr po obrazie. 

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

I uczymy sieć...

In [ ]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          validation_data=(x_val, y_val))

**Zad. 3.2: Podłącz sieć do Tensorboarda i wypróbuj innych parametrów lub archotektur**

### Fast.ai

Podobne (choć nie identyczne) podejście można zaimplementować w fast.ai. Na stronie [githubie fast.ai](https://github.com/fastai/fastai/blob/master/courses/dl1/lesson4-imdb.ipynb) można prześledzić jak stworzyć generator tekstu, którego wagi (embeddings) później zostaną wykorzystane do klasyfikacji. Sieci wykorzystane w tym przykładzie są inne, ale idea dwuetapowego rozwiązywania problemu analizy tekstu pozostaje. Przykład z fast.ai pokazuje znacznie silniejszy model, ale wymaga przy tym znacznie więcej czasu na wgryzienie się w kod.